In [ ]:
import sys
sys.path.append("../")

In [ ]:
import numpy as np
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C, WhiteKernel, Matern
from sklearn.gaussian_process import GaussianProcessRegressor 
from core import * 

import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [ ]:
import matplotlib.colors as mcolors
plt.rcParams.update({'font.size': 14})

In [ ]:
%load_ext autoreload
%autoreload 2

# Load true and res

In [ ]:
f_true_list = np.load('true_funcs/3d_RBF_4111_plot.npy', allow_pickle=True)
res = np.load('results/t10/3d_RBF_4111_Gaussian_plot.npy', 
              allow_pickle=True)

## Set input and grid

In [ ]:
tau_n, l_n = 4, np.ones(3)
w_n, sigma_n = np.zeros(3), np.eye(3)
domain = [[-5, 5], [-5, 5], [-5, 5]]
inputs = GaussianInputs(w_n, sigma_n, domain)
kernel = C(tau_n, 'fixed') * RBF(l_n, 'fixed')
sgp = GaussianProcessRegressor(kernel=kernel, random_state=0)

In [ ]:
bw = 0.1
x_grid = np.linspace(-5,  5,  51)
y_grid = np.linspace(-5,  5,  51)
z_grid = np.linspace(-5,  5,  51)
x_mesh, y_mesh, z_mesh = np.meshgrid(x_grid, y_grid, z_grid)
grids = np.concatenate((x_mesh.reshape(-1, 1), y_mesh.reshape(-1, 1), z_mesh.reshape(-1, 1)), axis=1)
pdf_grids = inputs.pdf(grids)

# Analyze one case

In [ ]:
function_num = 0
plot_3d(2.6, f_true_list[function_num], sgp, 
        res[function_num][0][0][0], res[function_num][0][1][0],
        (2.6, -3.4))

## Plot 3D isosurface 

In [ ]:
level = -6

for DX in (res[function_num][0][0][0], res[function_num][0][1][0]):
    sgp.fit(DX, f_true_list[function_num](DX))
    fig = go.Figure(go.Isosurface(
        x=x_mesh.flatten(),
        y=y_mesh.flatten(),
        z=z_mesh.flatten(),
        value=sgp.predict(grids),
        isomin=level,
        isomax=level,
        surface_count=1,
        opacity=0.8,
        caps=dict(x_show=True, y_show=True)
        ))
    fig.show()